In [1]:
## Questions

### Question 1

#Describe a commercial project in which you set up and analysed tracking data. What actions came out of this work?

## Question 2

#List the Top 10 authors by their average rating.

#Use any approach you feel comfortable with, but include any scripts with your answer and make sure your working is
#clear.

#What thoughts do you have on the results?

### Question 3

#Recommend 10 books similar to "The Golden Compass" by Philip Pullman.

#Again, you  may use any tools; include scripts and clear workings in your answer.


In [2]:
# TO RUN THIS, git clone the report into a folder and point PATH to the `data-scientist\data` subfolder.

PATH = r'C:\python37\reedsy\data-scientist\data' # TODO request this data from git so notebook works anywhere?
import pandas #import csv -- pandas.from_csv() is better as it preserves data types
import os
import glob
from pprint import pprint as pp

datasets = {}
for filename in glob.glob(PATH+'\*'):
    with open(os.path.join(PATH,filename),'r',encoding='utf8') as f:
        #fc = csv.DictReader(f)                
        basename = os.path.basename(filename)        
        datasets[basename] = pandas.read_csv(filename) # a cleaner importer than python's default 'csv' library
        print(basename, datasets[basename].keys())

books.csv Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')
book_tags.csv Index(['goodreads_book_id', 'tag_id', 'count'], dtype='object')
ratings.csv Index(['book_id', 'user_id', 'rating'], dtype='object')
tags.csv Index(['tag_id', 'tag_name'], dtype='object')
to_read.csv Index(['user_id', 'book_id'], dtype='object')


In [12]:
# Question 2: List the Top 10 authors by their average rating.
#
# data source: books.csv (best_book_id, average_rating, title)
# aggregate by authors
# - split multiple author books into separate records
# - filter by: at least 100 reviews
# - filter by: dedupe book_id so count editions of a book only once

def top_authors(min_ratings=100):
    # new data set with all books grouped by author
    from collections import defaultdict
    authors = defaultdict(list)
    for index, book in datasets['books.csv'].iterrows():
        if book['ratings_count'] < min_ratings:
            continue
        
        if ', ' in book['authors']:            
            for author in book['authors'].split(','): # split authors appart; preserve and duplicate ratings for each
                authors[author.strip()].append( book )
        else:
            authors[book['authors']].append( book )
    print(len(list(set(authors))),'unique authors')
    print(list(authors.keys())[:100])
    
    # grand mean: average rating per author, based on average rating per book with at least 100 ratings
    grand_means = []
    for author,books in authors.items():
        avg_rating = round(sum([float(book['average_rating']) for book in books])/len(books),2)
        grand_means.append( {'author': author, 'average_rating': avg_rating, 'books': len(books), 'meta': books} )
    return grand_means

grand_means = top_authors()
top10 = sorted(datasets['books.csv'].iterrows(), key=lambda x: float(x[1]['average_rating']), reverse=True)[:10]
print('top books')
pp([(i['best_book_id'], i['average_rating'], i['title']) for index,i in top10])

top10 = sorted(grand_means, key=lambda x: x['average_rating'], reverse=True)[:10]
print('top authors')
pp([(i['author'], i['average_rating'], i['books']) for i in top10])


5841 unique authors
['Suzanne Collins', 'J.K. Rowling', 'Mary GrandPré', 'Stephenie Meyer', 'Harper Lee', 'F. Scott Fitzgerald', 'John Green', 'J.R.R. Tolkien', 'J.D. Salinger', 'Dan Brown', 'Jane Austen', 'Khaled Hosseini', 'Veronica Roth', 'George Orwell', 'Erich Fromm', 'Celâl Üster', 'Anne Frank', 'Eleanor Roosevelt', 'B.M. Mooyaart-Doubleday', 'Stieg Larsson', 'Reg Keeland', 'Rufus Beck', 'Alice Sebold', 'William Golding', 'William Shakespeare', 'Robert           Jackson', 'Gillian Flynn', 'Kathryn Stockett', 'John Steinbeck', 'Arthur Golden', 'E.L. James', 'Paulo Coelho', 'Alan R. Clarke', 'Lois Lowry', 'C.S. Lewis', 'Audrey Niffenegger', 'George R.R. Martin', 'Elizabeth Gilbert', 'Rick Riordan', 'Louisa May Alcott', 'Charlotte Brontë', 'Michael Mason', 'Nicholas Sparks', 'Yann Martel', 'Sara Gruen', 'Markus Zusak', 'Ray Bradbury', 'Shel Silverstein', 'Cassandra Clare', 'Christopher Paolini', 'Douglas Adams', 'Aldous Huxley', 'Sue Monk Kidd', 'Mark Twain', 'John Seelye', 'Guy Car

In [4]:
# get _id and tags for golden compass
import time
for index,item in datasets['books.csv'].iterrows():
    if 'Golden Compass' in item['title']:
        #print( item )
        core_book_id = item['best_book_id']        
book_tags = datasets['book_tags.csv']
core_tags = set(book_tags.loc[book_tags['goodreads_book_id'] == core_book_id]['tag_id']) # Series --> set
tag_names = dict(datasets['tags.csv'].to_dict('split')['data'])
print('Golden Compass id:', core_book_id, 'tags:')
print(sorted([(tag, tag_names[tag]) for tag in core_tags],key=lambda x:x[0], reverse=True))

Golden Compass id: 119322 tags:
[(33165, 'youth'), (33153, 'youngadult'), (33124, 'young-adult-fiction'), (33121, 'young-adult-fantasy'), (33114, 'young-adult'), (33113, 'young'), (33023, 'ya-lit'), (33012, 'ya-fiction'), (33009, 'ya-fantasy'), (32996, 'ya-books'), (32989, 'ya'), (31617, 'urban-fantasy'), (30860, 'to-reread'), (30574, 'to-read'), (30573, 'to-re-read'), (30521, 'to-buy'), (29452, 'teen'), (29011, 'supernatural'), (28610, 'steampunk'), (28300, 'speculative-fiction'), (27199, 'series'), (26897, 'scifi-fantasy'), (26894, 'scifi'), (26842, 'science-fiction-fantasy'), (26837, 'science-fiction'), (26785, 'sci-fi-fantasy'), (26774, 'sci-fi-and-fantasy'), (26771, 'sci-fi'), (25765, 'reread'), (25647, 'religion'), (25151, 'read-in-2015'), (24960, 're-read'), (23442, 'philip-pullman'), (22973, 'paranormal'), (22753, 'owned-books'), (22743, 'owned'), (22689, 'own-it'), (22034, 'novels'), (21989, 'novel'), (20939, 'mystery'), (20849, 'my-library'), (20774, 'my-books'), (19983, 'mid

In [5]:
# prepare the book_tags lookup dictionary
from collections import Counter, defaultdict
useful = book_tags.to_dict('records')

In [7]:
# Question 3: Recommend 10 books similar to "The Golden Compass" by Philip Pullman.
#
# looking up books that have the most overlapping tags.
# - This ignores book ratings, which would be important for recommendations.
# - this ignores the COUNT of each tag - could be useful for filtering out less common tags and weighting others more
# - overall, the list is very similar to Golden Compass -- too similar in fact. Includes the same author's book series in top 4.

df = datasets['books.csv']
to_read = datasets['to_read.csv'] # Index(['user_id', 'book_id'], dtype='object')
similar = defaultdict(set) # book: set-overlapping-tags
for x in useful: # a million rows
    if x['tag_id'] in core_tags:
        similar[x['goodreads_book_id']].add(x['tag_id'])
       
recs = sorted(similar.items(), key=lambda x: len(x[1]), reverse=True) # book_id : set(tags)

print('top 10 books')
print([(df[df['best_book_id'] == x[0]]['title'],len(x[1])) for x in recs[:10]])

top 10 books
[(61    The Golden Compass (His Dark Materials, #1)
Name: title, dtype: object, 100), (367    The Subtle Knife (His Dark Materials, #2)
Name: title, dtype: object, 91), (479    The Amber Spyglass (His Dark Materials, #3)
Name: title, dtype: object, 90), (936    His Dark Materials (His Dark Materials #1-3)
Name: title, dtype: object, 78), (3062    The Dark Is Rising (The Dark is Rising, #2)
Name: title, dtype: object, 75), (116    A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
Name: title, dtype: object, 74), (222    Artemis Fowl (Artemis Fowl, #1)
Name: title, dtype: object, 74), (5462    Greenwitch (The Dark Is Rising, #3)
Name: title, dtype: object, 73), (1154    The Amulet of Samarkand (Bartimaeus, #1)
Name: title, dtype: object, 73), (1349    Lirael (Abhorsen, #2)
Name: title, dtype: object, 72)]


In [8]:
# BONUS: exclude same author from the list.
print('top 10 books')
found = 0
for x in recs:
    author = df[df['best_book_id'] == x[0]]['authors']        
    if ('Pullman' in author.values[0]) == False:
        print([found+1, df[df['best_book_id'] == x[0]]['title'], len(x[1])])
        found += 1
    if found == 10:
        break

top 10 books
[1, 3062    The Dark Is Rising (The Dark is Rising, #2)
Name: title, dtype: object, 75]
[2, 116    A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
Name: title, dtype: object, 74]
[3, 222    Artemis Fowl (Artemis Fowl, #1)
Name: title, dtype: object, 74]
[4, 5462    Greenwitch (The Dark Is Rising, #3)
Name: title, dtype: object, 73]
[5, 1154    The Amulet of Samarkand (Bartimaeus, #1)
Name: title, dtype: object, 73]
[6, 1349    Lirael (Abhorsen, #2)
Name: title, dtype: object, 72]
[7, 36    The Lion, the Witch, and the Wardrobe (Chronic...
Name: title, dtype: object, 72]
[8, 22    Harry Potter and the Chamber of Secrets (Harry...
Name: title, dtype: object, 71]
[9, 1195    A Wind in the Door (A Wrinkle in Time Quintet,...
Name: title, dtype: object, 71]
[10, 1297    The Golem's Eye (Bartimaeus, #2)
Name: title, dtype: object, 71]


In [9]:
# what if I sorted by books what most people have added to their "to_read" list?
# result: not as useful - these are the most popular somewhat similar books, but not the MOST similar ones.
# and the data about books on users' "to_read" lists is pretty sparse.

for rec in recs[:100]:
    # for new/obscure similary books: weight seldom read books more
    total_queued = to_read[to_read['book_id'] == rec[0]]['user_id'].count()
    #total_queued = to_read[to_read['book_id'] == rec[0]]['user_id'].count()
    if total_queued > 0:
        print(
            total_queued, len(rec[1]), 
            str(df[df['book_id'] == rec[0]]['title']),
            int(df[df['book_id'] == rec[0]]['ratings_4']),
            int(df[df['book_id'] == rec[0]]['ratings_5'])
        )
#print([(df[df['best_book_id'] == x[0]]['title'],len(x[1])) for x in recs[:20]])      

973 69 26    Harry Potter and the Half-Blood Prince (Harry ...
Name: title, dtype: object 459028 1161491
287 69 1    Harry Potter and the Sorcerer's Stone (Harry P...
Name: title, dtype: object 1156318 3011543
1293 69 17    Harry Potter and the Prisoner of Azkaban (Harr...
Name: title, dtype: object 509447 1266670
38 69 816    Howl's Moving Castle (Howl's Moving Castle, #1)
Name: title, dtype: object 45058 71962
1484 67 23    Harry Potter and the Goblet of Fire (Harry Pot...
Name: title, dtype: object 494926 1195045
12 66 1430    Redwall (Redwall, #1)
Name: title, dtype: object 29384 33140
89 64 6    The Hobbit
Name: title, dtype: object 665635 1119718


In [13]:
# What if I include the count of tags to filter out obscure ones? 
# rationale: tags are added to books by users, and if everyone is tagging a book the same way, it might mean something.
# this lookup has each book and returns a Counter dict of tags and counts for that book
# result: This is a very different list of somewhat similar books, but weighted more by popularity than similarity.

similar2 = defaultdict(Counter) # book: set-overlapping-tags
for x in useful: # a million rows
    if x['tag_id'] in core_tags: 
        similar2[x['goodreads_book_id']][x['tag_id']] = x['count']
    #if len(similar2) % 1000 == 0:
    #    print(len(similar2),'books')
    #    break
       
recs2 = sorted(similar2.items(), key=lambda x: sum([count for tag,count in x[1].most_common(6)]), reverse=True) # book_id : Counter(tag:count)

print('top 10 books')
pp(['{0}: tags: {1} count: {2}'.format(df[df['best_book_id'] == x[0]]['title'].values, len(x[1]), sum([count for tag,count in x[1].most_common(6)])) for x in recs2[:10]])

top 10 books
['["Harry Potter and the Sorcerer\'s Stone (Harry Potter, #1)"]: tags: 69 '
 'count: 664668',
 "['The Alchemist']: tags: 41 count: 646299",
 "['The Girl with the Dragon Tattoo (Millennium, #1)']: tags: 37 count: 636662",
 "['Les Misérables']: tags: 34 count: 576681",
 "['All the Light We Cannot See']: tags: 33 count: 563621",
 "['Cinder (The Lunar Chronicles, #1)']: tags: 49 count: 510381",
 "['Catch-22']: tags: 37 count: 504859",
 "['Slaughterhouse-Five']: tags: 47 count: 484893",
 "['Gone with the Wind']: tags: 38 count: 479669",
 "['A Tale of Two Cities']: tags: 36 count: 464400"]


In [11]:
# summary: I think the best quick "similar books" list sorts by number of shared tags from users, 
# excluding other books by the same author.
#
# for a truly effective recommendation function, I would weight by user ratings of these books and ask user if they want 
# more popular books or newer, more obscure books.
# I could also use a tag-edge network mapping algorithm to recommend books based on aspects of the book the user liked. 
# because not all books are good for the same reason, or appeal to people for the same reasons.

# note: evaluating these various sorts was easy because I had read most of these books. To optimize this without implicit knowledge,
# one would need to devise more tests, and user validation. This is usually the case.